In [32]:
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import yaml

from data_loader import get_dataloaders
from model import SRCNN
from test import evaluate_model
from utils import load_config, parse_args
import logging


In [33]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import numpy as np

In [34]:
resample_scale_factor = 2
num_epochs = 55
filename = "test_metrics.csv"

In [35]:
config = load_config("config.json")

In [36]:
_, _, testloader = get_dataloaders(seed=42, resample_scale_factor=resample_scale_factor)

2025-02-26 14:30:58,586 - INFO - Using 22 images from ../data/train2017
2025-02-26 14:30:58,588 - INFO - Train dataset size: 18
2025-02-26 14:30:58,589 - INFO - Validation dataset size: 2
2025-02-26 14:30:58,590 - INFO - Test dataset size: 2


In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
filename = "test_metrics_finetuned_models.csv"
for i in range(1,num_epochs+1):
    model = SRCNN()
    #if file exists
    if not os.path.isfile(f'../model/model_{i}_final.pt'):
        print("File not found!")
        continue
    model.load_state_dict(torch.load(f'../model/model_{i}_final.pt', map_location=device))
    evaluate_model(model, testloader, config, filename)

100%|██████████| 1/1 [00:00<00:00,  8.58it/s]


In [38]:
def validate_model(model, validloader, criterion, device, config):
    model.eval()  # Set model to evaluation mode
    total_loss = 0.0
    with torch.no_grad():  # Disable gradient calculation during validation
        for inputs, labels in validloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)  # Accumulate loss

    avg_loss = total_loss / len(validloader.dataset)
    logging.info(f"Validation Loss: {avg_loss:.4f}")

    validation_metrics = evaluate_model(model, validloader, config)
    logging.info(validation_metrics)
    return validation_metrics

In [ ]:
metric_file = "../results/test_metrics_finetuned_models.csv"

In [39]:
loss_values = np.loadtxt("../model/losses.csv", delimiter=",", usecols=0)
print(loss_values)

psnr_values = np.loadtxt("../results/evaluation_metrics_results.csv", usecols=1, skiprows=1, delimiter=",")
print(psnr_values)

ssim_values = np.loadtxt("../results/evaluation_metrics_results.csv", usecols=2, skiprows=1, delimiter=",")
print(ssim_values)

msssim_values = np.loadtxt("../results/evaluation_metrics_results.csv", usecols=3, skiprows=1, delimiter=",")
print(ssim_values)

[0.00184647 0.01761448 0.00240064 ... 0.00104934 0.00115241 0.00137482]
[25.9221 25.925  26.781  27.2264 27.4342 27.5319 27.5389 27.6357 27.7238
 27.7612 27.7862 27.814  27.8434 27.851  27.8877 26.494  27.9106 27.9163
 27.96   27.986  27.9968 28.0236 28.0284 28.0162 28.0766 28.0583 28.0966
 28.1283 28.1407 28.156  28.0193 28.1823 28.2041 28.2182 28.1747 28.1939
 28.2781 28.2432 28.3118 28.3229 28.3453 28.2899 28.3685 28.1324 28.3693
 28.4038 28.1734 28.411  28.4016 28.3987 28.4353 28.4226 28.4483 28.3412
 28.4259 28.4761 28.487  28.4932 28.4758 28.5072 28.4157 28.5189 28.5218
 28.5006 28.5055 28.5307 28.5478 28.3604 28.2444 28.5636 28.5651 28.5183
 28.5731 28.564  28.5872 28.594  28.5662 28.5966 28.5019 28.6101 28.6099
 28.53   28.6108 28.5947 28.6242 28.634  28.6328 28.6414 28.5647 28.644
 28.5516 28.6486 28.3987 28.6188 28.5972 28.6733 28.6556 28.6719 28.6815
 28.6549 28.6888 16.8248 30.8352 30.8352 30.8352 28.7882 28.7882 28.3346
 28.3469 28.3589 28.3684 28.3669 28.0045 28.4008 28.3

In [40]:
num_epochs = len(loss_values) #// 296
loss_x = np.arange(0, num_epochs + 1, 1 / 296)
loss_x = loss_x[:len(loss_values)]

num_epochs = len(psnr_values) 
psnr_x = np.arange(0, num_epochs + 1, 1 )
psnr_x = psnr_x[:len(psnr_values)]

num_epochs = len(ssim_values)
ssim_x = np.arange(0, num_epochs + 1, 1 )
ssim_x = ssim_x[:len(ssim_values)]